In [2]:
# Импортируем pandas для обработки данных

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Загружаем таблицу с информацией о клиентах

clients = pd.read_csv('clients2.csv')

clients.head(3)

,client_id,client_id.1,first_issue_date,first_redeem_date,age,gender
0,000012768d,NaN,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,NaN,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,00010925a5,NaN,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U


In [4]:
# Удалим колонку с пустыми значениями

clients = clients.drop('client_id.1', axis = 1)

In [5]:
# Загружаем таблицу с информацией о том, было ли воздействие маркетинговой компанией
# на клиента и была ли совершена покупка 

train = pd.read_csv('train.csv')

train.head(3)

,client_id,treatment_flg,purchased
0,ad6561e2d8,1,1
1,7c1ccbf93f,1,1
2,b58fadcab6,1,1


In [6]:
# Необходимо ввести таргетную переменную, которая равна:
# 1 - если было воздействи и клиент купил, не было воздействия и клиент не купил
# 0 - во всех остальных случаях
# т.е. применяем метод трансформации классов: Z = Y * W + (1 - Y) * (1 - W)

train['target'] = train['purchased'] * train['treatment_flg'] + (1 - train['purchased']) * (1 - train['treatment_flg'])

train.head(3)

,client_id,treatment_flg,purchased,target
0,ad6561e2d8,1,1,1
1,7c1ccbf93f,1,1,1
2,b58fadcab6,1,1,1


In [7]:
# Переводим столбцы с датами и временем в формат datetime

clients['first_issue_date'] = pd.to_datetime(clients['first_issue_date'])

clients['first_redeem_date'] = pd.to_datetime(clients['first_redeem_date'])

In [8]:
# Заменим пропуски в колонке с первой датой списания на последнбб дату в колонке

clients['first_redeem_date'].fillna(clients.first_redeem_date.max(), inplace=True)

In [9]:
# Извлекаем информацию о месяце и дне недели и добавляем в соответствующие колонки

clients['year_issue'] = clients.first_issue_date.dt.year
clients['month_issue'] = clients.first_issue_date.dt.month
clients['day_issue'] = clients.first_issue_date.dt.day

clients['year_redeem'] = clients.first_redeem_date.dt.year
clients['month_redeem'] = clients.first_redeem_date.dt.month
clients['day_redeem'] = clients.first_redeem_date.dt.day

clients.head(3)

,client_id,first_issue_date,first_redeem_date,age,gender,year_issue,month_issue,day_issue,year_redeem,month_redeem,day_redeem
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U,2017,8,5,2018,1,4
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F,2017,4,10,2017,4,23
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U,2018,7,24,2018,9,14


In [10]:
# Вычисляем разницу в днях между датой списания и выпуска

clients['days_between_issue_and_redeem'] = (clients['first_redeem_date'] - clients['first_issue_date']).dt.days

In [11]:
# Заменим средним выбросы в колонке возраст

q1, q3 = clients.age.quantile(0.25), clients.age.quantile(0.75)

iqr = q3 - q1

lower_bound = q1 - iqr
upper_bound = q3 + iqr

mask = (clients['age'] < lower_bound) | (clients['age'] > upper_bound)

mean = clients['age'].mean()

clients.loc[mask, 'age'] = mean

In [12]:
# Разобьем клиентов на возрастные категории

bins = [0, 18, 25, 35, 45, 55, 65, 100]

labels = ['age_<18', 'age_18-24', 'age_25-34', 'age_35-44', 'age_45-54', 'age_55-64', 'age_>=65']

clients['age_category'] = pd.cut(clients['age'], bins=bins, labels=labels, right = False)

clients.isna().sum()

client_id                        0
first_issue_date                 0
first_redeem_date                0
age                              0
gender                           0
year_issue                       0
month_issue                      0
day_issue                        0
year_redeem                      0
month_redeem                     0
day_redeem                       0
days_between_issue_and_redeem    0
age_category                     0
dtype: int64

In [13]:
# Разобьем клиентов на категории по времени владения картой покупателя

bins = [-50, 50, 100, 200, 1000]

labels = ['card_days_<50', 'card_days_50-99', 'card_days_100-199', 'card_days_>=200']

clients['card_owning'] = pd.cut(clients['days_between_issue_and_redeem'], bins=bins, labels=labels, right = False)

clients.isna().sum()

client_id                        0
first_issue_date                 0
first_redeem_date                0
age                              0
gender                           0
year_issue                       0
month_issue                      0
day_issue                        0
year_redeem                      0
month_redeem                     0
day_redeem                       0
days_between_issue_and_redeem    0
age_category                     0
card_owning                      0
dtype: int64

In [14]:
# Загружаем таблицу с информацией о покупках клиентов

train_purch = pd.read_csv('train_purch.csv')

train_purch.head(3)

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN


In [15]:
# Заменим нулями пропуски

train_purch['trn_sum_from_red'].fillna(0, inplace=True)

In [16]:
# Переводим колонку transaction_datetime в формат datetime

train_purch['transaction_datetime'] = pd.to_datetime(train_purch['transaction_datetime'])

In [17]:
# Загружаем таблицу с информацией о продуктах

products = pd.read_csv('products.csv')

products.head(3)

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.40,0,0
1,0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.68,0,0
2,0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.50,0,0


In [18]:
cols = ['level_1','level_2','level_3','level_4']
for column in cols:
    
    mode = products[column].mode()[0]
    
    products[column] = products[column].fillna(mode)
    
    products[column] = products[column].apply(lambda x: int(x, 16) / 10 ** 12)

In [19]:
# Устанавливаем айди продукта в качестве индекса

products = products.set_index('product_id')

## Проверим трейновые и тестовые данные на однородность с помощью метода adversarial validation.

Добавим в трейновую выборку колонку с "таргетом" 0, а в тестовую - с "таргетом" 1. 
Объединим два датафрейма и обучим модель классификации. Замерим качество модели с помощью ROC-AUC.
Если качество будет примерно равно 0,5, т.е. модель разделяет выборки случайно, значит они однородны.

In [20]:
# Загружаем таблицу с информацией о покупках клиентов, для которых делаем предсказание

test_purch = pd.read_csv('test_purch.csv')

test_purch.head(3)

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,a375a035b9,2.0,50.0,NaN
1,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,6f9e02ee36,1.0,90.0,NaN
2,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,76d287ce03,42e1eb8d23,1.0,20.0,NaN


In [21]:
# Заменим нулями пропуски в trn_sum_from_red, т.к. в трейне тоже заменяли

test_purch['trn_sum_from_red'].fillna(0, inplace=True)

In [22]:
# Копируем датасеты

train_AV = train_purch.copy()
test_AV = test_purch.copy()

# Добавляем колонки с "таргетами"

train_AV['AV_label'] = 0
test_AV['AV_label'] = 1

# Объединим два датасета

all_data = pd.concat([train_AV, test_AV], axis=0, ignore_index=True)

# Удалим лишние колонки

all_data = all_data.drop(['transaction_datetime', 'client_id', 'transaction_id'], axis=1)

# Перемешаем данные

all_data_shuffled = all_data.sample(frac=1)

# Разделим признаки и таргет

X_AV = all_data_shuffled.drop(['AV_label'], axis=1)
y_AV = all_data_shuffled['AV_label']

In [23]:
# Разделим данные на трейн и тест

from sklearn.model_selection import train_test_split



X_train_AV, X_test_AV, Y_train_AV, Y_test_AV = train_test_split(X_AV,
                                                                y_AV,
                                                                test_size=0.2,
                                                                random_state=9
                                                               )

In [24]:
# Вычислим отношение классов, чтобы учесть при обучении модели

len(train_AV[train_AV['AV_label'] == 0]) / (len(test_AV) + len(train_AV)), 
len(test_AV[test_AV['AV_label'] == 1]) / (len(test_AV) + len(train_AV))

0.30082730657977713

In [26]:
# Обучим catboost с учетом баланса классов

from catboost import CatBoostClassifier


cat_AV = CatBoostClassifier(cat_features = ['store_id', 'product_id'],
                            n_estimators = 100,
                            class_weights=[0.7, 0.3])

cat_AV.fit(X_train_AV, Y_train_AV)

Learning rate set to 0.5
0:	learn: 0.5142311	total: 5.02s	remaining: 8m 17s
1:	learn: 0.4596032	total: 7.48s	remaining: 6m 6s
2:	learn: 0.4283182	total: 10.7s	remaining: 5m 47s
3:	learn: 0.4161427	total: 14.1s	remaining: 5m 37s
4:	learn: 0.4113938	total: 17.6s	remaining: 5m 34s
5:	learn: 0.4095341	total: 19.5s	remaining: 5m 5s
6:	learn: 0.4088047	total: 21.7s	remaining: 4m 47s
7:	learn: 0.4084975	total: 24.1s	remaining: 4m 37s
8:	learn: 0.4083429	total: 25.9s	remaining: 4m 22s
9:	learn: 0.4082377	total: 28.6s	remaining: 4m 17s
10:	learn: 0.4081842	total: 31s	remaining: 4m 11s
11:	learn: 0.4081516	total: 33.5s	remaining: 4m 5s
12:	learn: 0.4081083	total: 36.5s	remaining: 4m 4s
13:	learn: 0.4080923	total: 39.6s	remaining: 4m 3s
14:	learn: 0.4080758	total: 41.9s	remaining: 3m 57s
15:	learn: 0.4080462	total: 45.5s	remaining: 3m 58s
16:	learn: 0.4080266	total: 48.5s	remaining: 3m 56s
17:	learn: 0.4079974	total: 51.3s	remaining: 3m 53s
18:	learn: 0.4079837	total: 54.4s	remaining: 3m 51s
19:	

In [27]:
# Вычислим значение ROC-AUC

from sklearn.metrics import roc_auc_score

print(f'ROC-AUC тренировочной выборки {roc_auc_score(Y_train_AV, cat_AV.predict_proba(X_train_AV)[:, 1])}')
print(f'ROC-AUC тестовой выборки {roc_auc_score(Y_test_AV, cat_AV.predict_proba(X_test_AV)[:, 1])}')

ROC-AUC тренировочной выборки 0.6726687376153501
ROC-AUC тестовой выборки 0.6703018534278841


Видно, что модель достаточно неплохо умеет разделять тренировочнкую и тестовую выборки. Посмотрим с помощью каких признаков

In [28]:
# Вызовем метод get_feature_importance, с помощью которого посмотрим на важность признаков

cat_AV.get_feature_importance(prettified=True)

,Feature Id,Importances
0,store_id,79.942620
1,purchase_sum,8.216231
2,regular_points_received,4.960821
3,regular_points_spent,2.821309
4,product_id,1.568855
5,trn_sum_from_iss,1.169012
6,express_points_spent,0.937508
7,express_points_received,0.204691
8,product_quantity,0.139766
9,trn_sum_from_red,0.039188


In [29]:
# Удалим признак store_id и заново обучим модель

X_AV_drop = X_AV.drop(['store_id'], axis=1)

# Заново сделаем разделение 

X_train_AV_drop, X_test_AV_drop, Y_train_AV, Y_test_AV = train_test_split(X_AV_drop,
                                                                y_AV,
                                                                test_size=0.2,
                                                                random_state=9
                                                               )

# Обучим модель

cat_AV_drop = CatBoostClassifier(cat_features = ['product_id'],
                            n_estimators = 100,
                            class_weights=[0.7, 0.3])

                                                 
cat_AV_drop.fit(X_train_AV_drop, Y_train_AV)

# Посчитаем ROC-AUC

print(f'ROC-AUC тренировочной выборки {roc_auc_score(Y_train_AV, cat_AV_drop.predict_proba(X_train_AV_drop)[:, 1])}')
print(f'ROC-AUC тестовой выборки {roc_auc_score(Y_test_AV, cat_AV_drop.predict_proba(X_test_AV_drop)[:, 1])}')                                               

Learning rate set to 0.5
0:	learn: 0.5142305	total: 3.37s	remaining: 5m 33s
1:	learn: 0.4596035	total: 5.53s	remaining: 4m 30s
2:	learn: 0.4410208	total: 7.23s	remaining: 3m 53s
3:	learn: 0.4350027	total: 9.27s	remaining: 3m 42s
4:	learn: 0.4331985	total: 10.9s	remaining: 3m 26s
5:	learn: 0.4326928	total: 12.5s	remaining: 3m 15s
6:	learn: 0.4325582	total: 15s	remaining: 3m 19s
7:	learn: 0.4324359	total: 18.5s	remaining: 3m 32s
8:	learn: 0.4324029	total: 22.2s	remaining: 3m 44s
9:	learn: 0.4323870	total: 24.4s	remaining: 3m 39s
10:	learn: 0.4323806	total: 25.9s	remaining: 3m 29s
11:	learn: 0.4323664	total: 27.3s	remaining: 3m 20s
12:	learn: 0.4323617	total: 28.8s	remaining: 3m 12s
13:	learn: 0.4323480	total: 31.1s	remaining: 3m 10s
14:	learn: 0.4323433	total: 32.6s	remaining: 3m 4s
15:	learn: 0.4323359	total: 34.1s	remaining: 2m 58s
16:	learn: 0.4323337	total: 35.5s	remaining: 2m 53s
17:	learn: 0.4323241	total: 36.8s	remaining: 2m 47s
18:	learn: 0.4323184	total: 38.4s	remaining: 2m 43s


Теперь модель классифицирует практически случайно, а значит выборки стали однородны. 

In [22]:
# Удаляем признак id_store из основных датасетов

train_purch = train_purch.drop(['store_id'], axis=1)
test_purch = test_purch.drop(['store_id'], axis=1)

In [110]:
# Создаем новую таблицу с датой первой и последней покупки для каждого клиента
# и разницу в днях между датами

purch_diff = train_purch.groupby('client_id')['transaction_datetime'].agg(['min', 'max'])

purch_diff['days_between_first_and_last_purchase'] = (purch_diff['max'] - purch_diff['min']).dt.days

purch_diff.head(3)

,min,max,days_between_first_and_last_purchase
client_id,,,
000012768d,2018-12-01 07:12:45,2019-03-14 15:01:47,103
000036f903,2018-11-28 10:48:36,2019-03-17 10:29:37,108
0001f552b0,2018-11-24 07:14:43,2019-03-16 10:54:02,112


In [24]:
# Объединяем таблицу с транзакциями с таблицей products

train_purch_prod = train_purch.merge(products, left_on = 'product_id', right_on = 'product_id')


train_purch_prod.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_id,product_quantity,...,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.00,9a80204f78,2.0,...,0.97611,1.018626,0.766042,0.349578,116.0,082560ca58,63243765ed,0.031,0,0
1,0027aac018,7be996a830,2018-12-08 14:46:14,1.0,0.0,-21.0,0.0,229.00,9a80204f78,1.0,...,0.97611,1.018626,0.766042,0.349578,116.0,082560ca58,63243765ed,0.031,0,0
2,00b19bc224,1001a6e6cb,2019-01-02 14:12:39,4.0,0.0,0.0,0.0,403.49,9a80204f78,1.0,...,0.97611,1.018626,0.766042,0.349578,116.0,082560ca58,63243765ed,0.031,0,0
3,00bb1bb4e8,c79c6f1815,2019-01-03 17:15:43,62.9,0.0,0.0,0.0,6893.00,9a80204f78,1.0,...,0.97611,1.018626,0.766042,0.349578,116.0,082560ca58,63243765ed,0.031,0,0
4,00e38f656d,c85b58be92,2019-03-15 14:37:08,32.5,0.0,-496.0,0.0,4480.00,9a80204f78,1.0,...,0.97611,1.018626,0.766042,0.349578,116.0,082560ca58,63243765ed,0.031,0,0


In [25]:
# Группируем по айди клиента, транзакции и вычисляем среднее

train_purch_prod_mean = train_purch_prod.groupby(['client_id', 'transaction_id'], as_index=False).mean()
train_purch_prod_mean

,client_id,transaction_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,level_1,level_2,level_3,level_4,segment_id,netto,is_own_trademark,is_alcohol
0,000012768d,6a0e96d0bc,8.0,0.0,0.0,0.0,803.00,0.812500,50.250000,0.0,0.925470,0.586136,0.542682,0.456485,112.562500,0.640312,0.062500,0.0
1,000012768d,7e3e2e3984,10.0,0.0,0.0,0.0,1007.00,1.105263,53.000000,0.0,0.919251,0.571095,0.560379,0.534957,134.631579,0.411526,0.105263,0.0
2,000012768d,b34f23306e,2.0,0.0,0.0,0.0,419.00,1.000000,69.833333,0.0,0.886083,0.639724,0.521049,0.443040,154.166667,0.633333,0.000000,0.0
3,000012768d,c1ca85d462,5.7,0.0,0.0,0.0,574.00,1.272727,52.272727,0.0,0.906012,0.738967,0.679766,0.494065,131.600000,0.566182,0.090909,0.0
4,000036f903,0a3d640bf4,7.0,0.0,0.0,0.0,700.00,1.000000,87.500000,0.0,0.891710,0.366123,0.432411,0.579088,114.000000,0.515000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814993,fffeca6d22,c8eb70dca7,0.4,0.0,0.0,0.0,97.00,1.000000,49.000000,0.0,0.841070,0.378947,0.658342,0.612132,44.000000,0.950000,0.000000,0.0
2814994,fffeca6d22,cb726dee92,0.5,0.0,0.0,0.0,109.00,1.500000,55.000000,0.0,0.841070,0.743754,0.869361,0.431939,148.500000,0.575000,0.000000,0.0
2814995,fffeca6d22,ce55437ec7,8.7,0.0,0.0,0.0,876.00,1.363636,79.727273,0.0,0.955117,0.864353,0.434827,0.458000,57.000000,0.503909,0.000000,0.0
2814996,fffeca6d22,cef49407f1,6.0,0.0,0.0,0.0,129.00,1.000000,32.500000,0.0,0.918380,0.685309,0.488448,0.624998,75.500000,0.394500,0.000000,0.0


In [26]:
# Суммируем сгруппированные данные

train_purch_prod_mean_sum = train_purch_prod_mean.groupby(['client_id'], as_index=False).sum()
train_purch_prod_mean_sum

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,level_1,level_2,level_3,level_4,segment_id,netto,is_own_trademark,is_alcohol
0,000012768d,25.7,0.0,0.0,0.0,2803.00,4.190490,225.356061,0.000000,3.636815,2.535921,2.303875,1.928547,532.960746,2.251354,0.258672,0.000000
1,000036f903,54.9,60.0,0.0,0.0,9805.00,33.020238,1908.035317,0.000000,29.056028,13.939773,13.185150,16.538514,5295.828571,14.782695,2.939683,0.142857
2,0001f552b0,78.9,0.0,0.0,0.0,6155.18,17.168651,1064.124603,0.000000,13.815457,7.805606,7.363509,8.994472,2142.655952,8.479519,1.546825,0.000000
3,00020e7b18,286.1,0.0,-592.0,-30.0,25819.61,27.175927,1582.721300,670.989519,16.566607,10.314484,10.896564,9.476811,2430.543222,7.740633,2.593499,0.000000
4,000220a0a7,104.1,0.0,0.0,0.0,8606.57,24.090643,2047.691292,0.000000,14.162958,5.429903,8.473054,7.236260,2216.772799,7.127940,2.152464,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140022,fffd5cd0c6,35.1,0.0,0.0,0.0,5731.63,21.106385,1143.125361,0.000000,16.891387,8.959518,7.885612,9.434981,2389.528139,11.974131,4.006421,0.000000
140023,fffd63dfe3,6.5,0.0,0.0,0.0,1340.51,5.133333,214.233333,0.000000,3.695877,2.621051,2.408306,2.057752,514.877778,1.368222,1.600000,0.444444
140024,fffd8c9d7d,11.0,0.0,0.0,0.0,1636.53,7.369048,462.797619,0.000000,4.565551,1.515058,1.603816,2.376105,656.369048,2.584512,1.285714,0.000000
140025,fffe0abb97,22.2,0.0,-26.0,-150.0,2716.42,10.892857,494.559524,336.833333,8.465962,4.028723,5.774497,3.484222,1329.874459,7.449494,0.250000,0.000000


In [27]:
# Удалим лишние колонки

train_purch_prod_mean_sum = train_purch_prod_mean_sum.drop(['level_1','level_2','level_3',
                                                            'level_4','segment_id',
                                                            'is_own_trademark', 'is_alcohol'], axis = 1)

In [28]:
# Соберем дополнительные агрегаты

train_purch_prod_agg = pd.DataFrame({'n_transactions': train_purch.groupby('client_id')['transaction_id'].nunique(),
                                     'n_products': train_purch.groupby('client_id')['product_quantity'].sum(),
                                     'n_segments': train_purch_prod.groupby('client_id')['segment_id'].nunique(),
                                     'n_brands': train_purch_prod.groupby('client_id')['brand_id'].nunique(),
                                     'n_vendors': train_purch_prod.groupby('client_id')['vendor_id'].nunique(),
                                     'level_1': train_purch_prod.groupby('client_id')['level_1'].mean(),
                                     'level_2': train_purch_prod.groupby('client_id')['level_2'].mean(),
                                     'level_3': train_purch_prod.groupby('client_id')['level_3'].mean(),
                                     'level_4': train_purch_prod.groupby('client_id')['level_4'].mean(),
                                     'segment_id': train_purch_prod.groupby('client_id')['segment_id'].mean(),
                                     'own_trademark_pct': train_purch_prod_mean.groupby('client_id')['is_own_trademark'].mean(),
                                     'alcohol_pct': train_purch_prod_mean.groupby('client_id')['is_alcohol'].mean()
                                    })

train_purch_prod_agg

,n_transactions,n_products,n_segments,n_brands,n_vendors,level_1,level_2,level_3,level_4,segment_id,own_trademark_pct,alcohol_pct
client_id,,,,,,,,,,,,
000012768d,4,54.0,23,28,29,0.914537,0.619153,0.575650,0.491556,129.411765,0.064668,0.000000
000036f903,32,169.0,41,51,44,0.903830,0.420932,0.407353,0.532473,164.050314,0.091865,0.004464
0001f552b0,15,106.0,40,46,42,0.933054,0.567186,0.499903,0.559315,150.464286,0.103122,0.000000
00020e7b18,18,394.0,44,86,69,0.909166,0.541710,0.581307,0.500956,141.596226,0.144083,0.000000
000220a0a7,15,140.0,36,52,46,0.943922,0.447351,0.526774,0.497031,147.500000,0.143498,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...
fffd5cd0c6,18,119.0,40,42,34,0.930588,0.542806,0.460741,0.482857,144.535354,0.222579,0.000000
fffd63dfe3,4,31.0,13,13,12,0.927495,0.626203,0.604979,0.527285,118.291667,0.400000,0.111111
fffd8c9d7d,5,27.0,13,9,9,0.917300,0.385661,0.362850,0.425527,127.000000,0.257143,0.000000


In [29]:
# Объединим таблицу с агрегатами с таблицей продуктов и транзакций

train_purch_agg = train_purch_prod_mean_sum.merge(train_purch_prod_agg, left_on = 'client_id', right_on = 'client_id')
train_purch_agg.head()

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,netto,...,n_segments,n_brands,n_vendors,level_1,level_2,level_3,level_4,segment_id,own_trademark_pct,alcohol_pct
0,000012768d,25.7,0.0,0.0,0.0,2803.00,4.190490,225.356061,0.000000,2.251354,...,23,28,29,0.914537,0.619153,0.575650,0.491556,129.411765,0.064668,0.000000
1,000036f903,54.9,60.0,0.0,0.0,9805.00,33.020238,1908.035317,0.000000,14.782695,...,41,51,44,0.903830,0.420932,0.407353,0.532473,164.050314,0.091865,0.004464
2,0001f552b0,78.9,0.0,0.0,0.0,6155.18,17.168651,1064.124603,0.000000,8.479519,...,40,46,42,0.933054,0.567186,0.499903,0.559315,150.464286,0.103122,0.000000
3,00020e7b18,286.1,0.0,-592.0,-30.0,25819.61,27.175927,1582.721300,670.989519,7.740633,...,44,86,69,0.909166,0.541710,0.581307,0.500956,141.596226,0.144083,0.000000
4,000220a0a7,104.1,0.0,0.0,0.0,8606.57,24.090643,2047.691292,0.000000,7.127940,...,36,52,46,0.943922,0.447351,0.526774,0.497031,147.500000,0.143498,0.066667


In [30]:
# Добавим таблицу с разницей между первой и последней покупкой

train_purch_agg_diff = train_purch_agg.merge(purch_diff, left_on = 'client_id', right_on = 'client_id')
train_purch_agg_diff.head()

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,netto,...,level_1,level_2,level_3,level_4,segment_id,own_trademark_pct,alcohol_pct,min,max,days_between_first_and_last_purchase
0,000012768d,25.7,0.0,0.0,0.0,2803.00,4.190490,225.356061,0.000000,2.251354,...,0.914537,0.619153,0.575650,0.491556,129.411765,0.064668,0.000000,2018-12-01 07:12:45,2019-03-14 15:01:47,103
1,000036f903,54.9,60.0,0.0,0.0,9805.00,33.020238,1908.035317,0.000000,14.782695,...,0.903830,0.420932,0.407353,0.532473,164.050314,0.091865,0.004464,2018-11-28 10:48:36,2019-03-17 10:29:37,108
2,0001f552b0,78.9,0.0,0.0,0.0,6155.18,17.168651,1064.124603,0.000000,8.479519,...,0.933054,0.567186,0.499903,0.559315,150.464286,0.103122,0.000000,2018-11-24 07:14:43,2019-03-16 10:54:02,112
3,00020e7b18,286.1,0.0,-592.0,-30.0,25819.61,27.175927,1582.721300,670.989519,7.740633,...,0.909166,0.541710,0.581307,0.500956,141.596226,0.144083,0.000000,2018-11-23 08:21:26,2019-03-15 12:04:23,112
4,000220a0a7,104.1,0.0,0.0,0.0,8606.57,24.090643,2047.691292,0.000000,7.127940,...,0.943922,0.447351,0.526774,0.497031,147.500000,0.143498,0.066667,2018-12-11 16:54:16,2019-03-12 16:44:43,90


In [31]:
# Добавим таблицу с информацией о клиентах

train_purch_agg_diff_cli = train_purch_agg_diff.merge(clients, left_on = 'client_id', right_on = 'client_id')
train_purch_agg_diff_cli.head()

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,netto,...,gender,year_issue,month_issue,day_issue,year_redeem,month_redeem,day_redeem,days_between_issue_and_redeem,age_category,card_owning
0,000012768d,25.7,0.0,0.0,0.0,2803.00,4.190490,225.356061,0.000000,2.251354,...,U,2017,8,5,2018,1,4,152,age_45-54,card_days_100-199
1,000036f903,54.9,60.0,0.0,0.0,9805.00,33.020238,1908.035317,0.000000,14.782695,...,F,2017,4,10,2017,4,23,12,age_>=65,card_days_<50
2,0001f552b0,78.9,0.0,0.0,0.0,6155.18,17.168651,1064.124603,0.000000,8.479519,...,F,2017,6,30,2018,8,28,423,age_25-34,card_days_>=200
3,00020e7b18,286.1,0.0,-592.0,-30.0,25819.61,27.175927,1582.721300,670.989519,7.740633,...,U,2017,11,27,2018,1,10,44,age_>=65,card_days_<50
4,000220a0a7,104.1,0.0,0.0,0.0,8606.57,24.090643,2047.691292,0.000000,7.127940,...,M,2017,12,9,2018,10,4,298,age_45-54,card_days_>=200


In [32]:
# Добавим колонку с резницей в днях между выпуском карты и первой покупкой

train_purch_agg_diff_cli['days_after_first_issue_date'] = (train_purch_agg_diff_cli['min'] - train_purch_agg_diff_cli['first_issue_date']).dt.days

In [54]:
# Пропуски в сегмент айди заменим модой

mode = train_purch_agg_diff_cli['segment_id'].mode()[0]

train_purch_agg_diff_cli['segment_id'] = train_purch_agg_diff_cli['segment_id'].fillna(mode)

In [56]:
# Добавим информацию о таргете

train_purch_agg_diff_cli_target = train.merge(train_purch_agg_diff_cli, left_on = 'client_id', right_on = 'client_id')
train_purch_agg_diff_cli_target.head()

,client_id,treatment_flg,purchased,target,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,...,year_issue,month_issue,day_issue,year_redeem,month_redeem,day_redeem,days_between_issue_and_redeem,age_category,card_owning,days_after_first_issue_date
0,ad6561e2d8,1,1,1,97.3,0.0,-64.0,0.0,10801.41,40.341497,...,2017,12,3,2018,5,23,171,age_45-54,card_days_100-199,354
1,7c1ccbf93f,1,1,1,9.4,0.0,0.0,0.0,851.00,2.606061,...,2017,11,10,2018,2,22,104,age_18-24,card_days_100-199,465
2,b58fadcab6,1,1,1,91.5,0.0,-309.0,0.0,14171.77,36.544877,...,2017,11,2,2018,9,15,316,age_35-44,card_days_>=200,384
3,e99e6fabb9,0,0,1,13.6,0.0,0.0,0.0,2872.00,14.642857,...,2018,5,16,2018,5,30,14,age_>=65,card_days_<50,200
4,27fb6f8520,1,1,1,82.2,20.0,-4.0,-50.0,9886.21,41.545815,...,2017,7,17,2017,12,15,150,age_25-34,card_days_100-199,503


In [57]:
# Удалим ненужные колонки и установим в качестве индекса client_id

train_features = train_purch_agg_diff_cli_target.drop(['treatment_flg', 'purchased', 'min',
                                                      'max', 'first_issue_date', 'first_redeem_date',
                                                      ], axis = 1)

train_features = train_features.set_index('client_id')

In [58]:
# Разделим данные на признаки и таргеты

X = train_features.drop('target', axis=1)

y = train_features.target

In [59]:
# Подготовим категориальные колонки для OneHotEncoding

numeric_columns = list(X.select_dtypes(exclude=['category', 'object']).columns)
category_columns = list(X.select_dtypes(include=['category', 'object']).columns)


numeric_columns_ind = [list(X.columns).index(col) for col in numeric_columns]
columns_ohe_ind = [list(X.columns).index(col) for col in category_columns]

In [103]:
# Создадим трансформер, который будет нормализовывать данные и закодирует категориальные колонки

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders.one_hot import OneHotEncoder


transformer = [
    ('StandardScaler', StandardScaler(), numeric_columns_ind),
    ('OneHotEncoder', OneHotEncoder(), columns_ohe_ind)
]

columns_transformer = ColumnTransformer(transformers=transformer)

In [75]:
# Загрузим модель случайного леса, параметры которого были подобраны через grid search

from sklearn.ensemble import RandomForestClassifier

rand_forest = RandomForestClassifier(n_estimators = 5000,
                                     max_depth = 5,
                                     random_state = 42,
                                     n_jobs = -1)

In [76]:
# Создадим пайплайн обработки признаков и обучения модели на всей выборке

from sklearn.pipeline import Pipeline


pipe_rf = Pipeline([('columns_transformer', columns_transformer),
                     ('rand_forest', RandomForestClassifier())])

pipe_rf.fit(X, y)

Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22, 24,
                                                   25, 26, 27, 28, 29, 30, ...]),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  [23, 31, 32])])),
                ('rand_forest', RandomForestClassifier())])

In [63]:
# Загружаем таблицу с id покупателей для теста

test = pd.read_csv('test.csv')

test.head()

,client_id
0,a9a604ed6e
1,ebd7360016
2,908cd9b8e8
3,dceb8ce861
4,f4f0ac6b06


In [64]:
# Переводим колонку transaction_datetime в формат datetime

test_purch['transaction_datetime'] = pd.to_datetime(test_purch['transaction_datetime'])

In [65]:
# Создаем новую таблицу с датой первой и последней покупки для каждого клиента
# и разницу в днях между датами

purch_diff_test = test_purch.groupby('client_id')['transaction_datetime'].agg(['min', 'max'])

purch_diff_test['days_between_first_and_last_purchase'] = (purch_diff_test['max'] - purch_diff_test['min']).dt.days

purch_diff_test.head(3)

,min,max,days_between_first_and_last_purchase
client_id,,,
00010925a5,2018-11-25 10:56:18,2019-03-08 07:09:50,102
00035a21d9,2019-02-09 11:46:22,2019-03-02 12:18:35,21
00038f9200,2018-12-07 11:03:50,2019-03-18 12:48:40,101


In [66]:
# Объединяем таблицу с транзакциями с таблицей products

test_purch_prod = test_purch.merge(products, left_on = 'product_id', right_on = 'product_id')


test_purch_prod.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_id,product_quantity,...,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,00010925a5,e91ffe5b50,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.00,a375a035b9,2.0,...,0.97611,1.018626,0.031802,1.065543,14.0,ab230258e9,57ee56f098,0.36,0,0
1,00674cdbcd,da33ffe1c2,2018-12-15 08:24:58,2.3,0.0,0.0,0.0,476.00,a375a035b9,1.0,...,0.97611,1.018626,0.031802,1.065543,14.0,ab230258e9,57ee56f098,0.36,0,0
2,00674cdbcd,3c53d65157,2019-01-17 09:40:16,1.0,0.0,0.0,0.0,39.00,a375a035b9,1.0,...,0.97611,1.018626,0.031802,1.065543,14.0,ab230258e9,57ee56f098,0.36,0,0
3,008815a235,9b26e8ebd9,2019-03-10 16:27:26,27.0,0.0,-63.0,0.0,2765.21,a375a035b9,1.0,...,0.97611,1.018626,0.031802,1.065543,14.0,ab230258e9,57ee56f098,0.36,0,0
4,0096a2720b,84c63e4bc3,2018-11-24 12:38:25,2.3,0.0,0.0,0.0,467.00,a375a035b9,1.0,...,0.97611,1.018626,0.031802,1.065543,14.0,ab230258e9,57ee56f098,0.36,0,0


In [67]:
# Группируем по айди клиента, транзакции

test_purch_prod_mean = test_purch_prod.groupby(['client_id', 'transaction_id'], as_index=False).mean()
test_purch_prod_mean

,client_id,transaction_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,level_1,level_2,level_3,level_4,segment_id,netto,is_own_trademark,is_alcohol
0,00010925a5,081dc6d998,2.3,0.0,0.0,0.0,478.0,0.857143,68.285714,0.0,0.860361,0.701703,0.840766,0.447873,120.500000,0.561429,0.142857,0.000000
1,00010925a5,09af961a1c,2.3,0.0,0.0,0.0,463.0,1.000000,92.600000,0.0,0.868078,0.406482,0.726560,0.371486,131.600000,0.530000,0.000000,0.000000
2,00010925a5,144898073a,1.4,0.0,0.0,0.0,299.0,0.714286,42.857143,0.0,0.918235,0.385775,0.614892,0.534524,119.428571,0.507143,0.428571,0.000000
3,00010925a5,1721d9d308,1.2,0.0,0.0,0.0,250.0,1.000000,125.000000,0.0,0.841070,0.425057,0.615463,0.626830,148.000000,1.000000,0.500000,0.000000
4,00010925a5,1c4bd83e38,5.8,0.0,0.0,0.0,581.0,1.000000,72.750000,0.0,0.913485,0.567544,0.484122,0.471887,189.857143,0.504125,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209946,fffff6ce77,e9182e5887,0.7,0.0,0.0,0.0,157.0,3.500000,79.000000,0.0,0.841070,0.743754,0.922669,0.302265,43.000000,0.487500,0.000000,0.000000
1209947,fffff6ce77,eba2694c0c,1.8,0.0,0.0,0.0,446.0,1.111111,49.555556,0.0,0.916092,0.412134,0.601988,0.393322,150.625000,0.492222,0.555556,0.111111
1209948,fffff6ce77,f0af5bcc0b,0.9,0.0,0.0,0.0,183.0,1.000000,61.000000,0.0,0.931096,0.706205,0.416822,0.368565,40.000000,0.800000,0.000000,0.000000
1209949,fffff6ce77,f58417b669,20.7,0.0,0.0,0.0,2077.0,1.550000,103.850000,0.0,0.883540,0.492175,0.514433,0.505062,161.526316,0.470800,0.050000,0.000000


In [68]:
# Суммируем сгруппированные датасет

test_purch_prod_mean_sum = test_purch_prod_mean.groupby(['client_id'], as_index=False).sum()
test_purch_prod_mean_sum

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,level_1,level_2,level_3,level_4,segment_id,netto,is_own_trademark,is_alcohol
0,00010925a5,31.8,0.0,-17.0,0.0,5883.00,18.388095,1505.292857,87.600000,15.713246,8.422317,11.114656,8.996209,2577.569048,11.601530,2.604762,0.500000
1,00035a21d9,23.6,0.0,-8.0,-30.0,1791.94,2.769231,201.642308,60.538462,2.784721,1.673245,1.857901,1.773093,340.524725,1.238100,0.125000,0.000000
2,00038f9200,82.8,0.0,-29.0,0.0,8385.00,56.593651,2693.201587,114.000000,43.432164,27.089532,28.378073,23.659196,6598.794444,28.098113,3.823016,0.000000
3,0004315e57,67.8,0.0,-39.0,-60.0,7590.00,16.608333,921.805664,150.622222,13.290077,6.787362,6.747269,6.265925,2034.026118,4.621756,2.059398,0.000000
4,0006fca4bf,35.4,0.0,0.0,0.0,2141.00,3.576923,287.192308,0.000000,2.905163,1.422704,1.360941,1.215018,334.884615,0.633846,0.153846,0.038462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60007,fff9aa288c,42.1,0.0,-46.0,0.0,4942.11,10.448120,838.267669,96.000000,9.241673,6.530684,5.847031,6.249204,1805.715873,3.240860,0.786466,1.471429
60008,fff9f772f6,95.9,0.0,0.0,0.0,6255.26,5.687302,253.465533,0.000000,3.707368,2.651725,2.480568,2.183158,608.480272,1.743007,1.050340,0.000000
60009,fffe0ed719,69.9,0.0,0.0,0.0,10417.48,42.666667,1844.558730,0.000000,28.241327,12.464133,15.580071,12.597902,4326.309890,14.041325,5.889683,0.000000
60010,fffea1204c,26.1,0.0,-40.0,0.0,4297.12,28.545238,1183.497619,58.750000,15.525537,8.798770,7.822567,8.057921,2280.683333,7.501202,0.000000,0.000000


In [69]:
# Удалим лишние колонки

test_purch_prod_mean_sum = test_purch_prod_mean_sum.drop(['level_1','level_2','level_3',
                                                            'level_4','segment_id',
                                                            'is_own_trademark', 'is_alcohol'], axis = 1)

In [70]:
# Соберем дополнительные агрегаты

test_purch_prod_agg = pd.DataFrame({'n_transactions': test_purch.groupby('client_id')['transaction_id'].nunique(),
                                     'n_products': test_purch.groupby('client_id')['product_quantity'].sum(),
                                     'n_segments': test_purch_prod.groupby('client_id')['segment_id'].nunique(),
                                     'n_brands': test_purch_prod.groupby('client_id')['brand_id'].nunique(),
                                     'n_vendors': test_purch_prod.groupby('client_id')['vendor_id'].nunique(),
                                     'level_1': test_purch_prod.groupby('client_id')['level_1'].mean(),
                                     'level_2': test_purch_prod.groupby('client_id')['level_2'].mean(),
                                     'level_3': test_purch_prod.groupby('client_id')['level_3'].mean(),
                                     'level_4': test_purch_prod.groupby('client_id')['level_4'].mean(),
                                     'segment_id': test_purch_prod.groupby('client_id')['segment_id'].mean(),
                                     'own_trademark_pct': test_purch_prod_mean.groupby('client_id')['is_own_trademark'].mean(),
                                     'alcohol_pct': test_purch_prod_mean.groupby('client_id')['is_alcohol'].mean()
                                    })

test_purch_prod_agg

,n_transactions,n_products,n_segments,n_brands,n_vendors,level_1,level_2,level_3,level_4,segment_id,own_trademark_pct,alcohol_pct
client_id,,,,,,,,,,,,
00010925a5,18,79.0,31,27,24,0.877929,0.472808,0.623640,0.475755,137.480000,0.144709,0.027778
00035a21d9,3,23.0,14,16,16,0.932377,0.562146,0.648907,0.553044,107.000000,0.041667,0.000000
00038f9200,48,186.0,41,49,40,0.905060,0.585070,0.579524,0.482871,142.363636,0.079646,0.000000
0004315e57,14,131.0,39,62,52,0.945936,0.491355,0.462914,0.459754,157.775862,0.147100,0.000000
0006fca4bf,3,43.0,20,16,15,0.919634,0.478630,0.419776,0.531851,138.642857,0.051282,0.012821
...,...,...,...,...,...,...,...,...,...,...,...,...
fff9aa288c,10,68.0,27,29,29,0.929415,0.668306,0.630362,0.654682,178.841270,0.078647,0.147143
fff9f772f6,4,129.0,39,38,39,0.922219,0.648145,0.643303,0.538361,149.175258,0.262585,0.000000
fffe0ed719,30,217.0,36,52,52,0.945945,0.391922,0.480976,0.410591,143.910828,0.196323,0.000000


In [71]:
# Объединим таблицу с агрегатами с таблицей продуктов и транзакций

test_purch_agg = test_purch_prod_mean_sum.merge(test_purch_prod_agg, left_on = 'client_id', right_on = 'client_id')
test_purch_agg.head()

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,netto,...,n_segments,n_brands,n_vendors,level_1,level_2,level_3,level_4,segment_id,own_trademark_pct,alcohol_pct
0,00010925a5,31.8,0.0,-17.0,0.0,5883.00,18.388095,1505.292857,87.600000,11.601530,...,31,27,24,0.877929,0.472808,0.623640,0.475755,137.480000,0.144709,0.027778
1,00035a21d9,23.6,0.0,-8.0,-30.0,1791.94,2.769231,201.642308,60.538462,1.238100,...,14,16,16,0.932377,0.562146,0.648907,0.553044,107.000000,0.041667,0.000000
2,00038f9200,82.8,0.0,-29.0,0.0,8385.00,56.593651,2693.201587,114.000000,28.098113,...,41,49,40,0.905060,0.585070,0.579524,0.482871,142.363636,0.079646,0.000000
3,0004315e57,67.8,0.0,-39.0,-60.0,7590.00,16.608333,921.805664,150.622222,4.621756,...,39,62,52,0.945936,0.491355,0.462914,0.459754,157.775862,0.147100,0.000000
4,0006fca4bf,35.4,0.0,0.0,0.0,2141.00,3.576923,287.192308,0.000000,0.633846,...,20,16,15,0.919634,0.478630,0.419776,0.531851,138.642857,0.051282,0.012821


In [72]:
# Добавим таблицу с разницей между первой и последней покупкой

test_purch_agg_diff = test_purch_agg.merge(purch_diff_test, left_on = 'client_id', right_on = 'client_id')
test_purch_agg_diff.head()

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,netto,...,level_1,level_2,level_3,level_4,segment_id,own_trademark_pct,alcohol_pct,min,max,days_between_first_and_last_purchase
0,00010925a5,31.8,0.0,-17.0,0.0,5883.00,18.388095,1505.292857,87.600000,11.601530,...,0.877929,0.472808,0.623640,0.475755,137.480000,0.144709,0.027778,2018-11-25 10:56:18,2019-03-08 07:09:50,102
1,00035a21d9,23.6,0.0,-8.0,-30.0,1791.94,2.769231,201.642308,60.538462,1.238100,...,0.932377,0.562146,0.648907,0.553044,107.000000,0.041667,0.000000,2019-02-09 11:46:22,2019-03-02 12:18:35,21
2,00038f9200,82.8,0.0,-29.0,0.0,8385.00,56.593651,2693.201587,114.000000,28.098113,...,0.905060,0.585070,0.579524,0.482871,142.363636,0.079646,0.000000,2018-12-07 11:03:50,2019-03-18 12:48:40,101
3,0004315e57,67.8,0.0,-39.0,-60.0,7590.00,16.608333,921.805664,150.622222,4.621756,...,0.945936,0.491355,0.462914,0.459754,157.775862,0.147100,0.000000,2018-11-25 13:20:56,2019-03-08 11:33:37,102
4,0006fca4bf,35.4,0.0,0.0,0.0,2141.00,3.576923,287.192308,0.000000,0.633846,...,0.919634,0.478630,0.419776,0.531851,138.642857,0.051282,0.012821,2019-02-20 07:47:30,2019-03-10 06:56:52,17


In [73]:
# Добавим таблицу с информацией о клиентах

test_purch_agg_diff_cli = test_purch_agg_diff.merge(clients, left_on = 'client_id', right_on = 'client_id')
test_purch_agg_diff_cli.head()

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,product_quantity,trn_sum_from_iss,trn_sum_from_red,netto,...,gender,year_issue,month_issue,day_issue,year_redeem,month_redeem,day_redeem,days_between_issue_and_redeem,age_category,card_owning
0,00010925a5,31.8,0.0,-17.0,0.0,5883.00,18.388095,1505.292857,87.600000,11.601530,...,U,2018,7,24,2018,9,14,51,age_>=65,card_days_50-99
1,00035a21d9,23.6,0.0,-8.0,-30.0,1791.94,2.769231,201.642308,60.538462,1.238100,...,U,2019,2,9,2019,3,2,21,age_>=65,card_days_<50
2,00038f9200,82.8,0.0,-29.0,0.0,8385.00,56.593651,2693.201587,114.000000,28.098113,...,U,2018,12,7,2019,1,12,35,age_>=65,card_days_<50
3,0004315e57,67.8,0.0,-39.0,-60.0,7590.00,16.608333,921.805664,150.622222,4.621756,...,U,2018,11,25,2019,2,2,68,age_35-44,card_days_50-99
4,0006fca4bf,35.4,0.0,0.0,0.0,2141.00,3.576923,287.192308,0.000000,0.633846,...,U,2019,2,20,2019,2,20,0,age_55-64,card_days_<50


In [82]:
# Добавим колонку с резницей в днях между выпуском карты и первой покупкой

test_purch_agg_diff_cli['days_after_first_issue_date'] = (test_purch_agg_diff_cli['min'] - test_purch_agg_diff_cli['first_issue_date']).dt.days

In [87]:
# Пропуски в сегмент айди заменим модой

mode = test_purch_agg_diff_cli['segment_id'].mode()[0]

test_purch_agg_diff_cli['segment_id'] = test_purch_agg_diff_cli['segment_id'].fillna(mode)

In [88]:
test_features = test_purch_agg_diff_cli.drop(['min', 'max',
                                              'first_issue_date',
                                              'first_redeem_date'
                                             ], axis = 1)

test_features = test_features.set_index('client_id')

In [105]:
pred = pipe_log.predict_proba(test_features)
pred

array([[0.46559598, 0.53440402],
       [0.44825131, 0.55174869],
       [0.47092594, 0.52907406],
       ...,
       [0.48794177, 0.51205823],
       [0.46418743, 0.53581257],
       [0.49740508, 0.50259492]])

In [106]:
total = test.copy()

In [107]:
total = test.copy()
total['pred'] = pred[:, 1]
total = total.set_index('client_id')
total.head(3)

,pred
client_id,
a9a604ed6e,0.534404
ebd7360016,0.551749
908cd9b8e8,0.529074


In [109]:
total.to_csv('/Users/vadimbaev/startml/Uplift/Totals/log_total.csv')

In [95]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier(eval_metric = 'NormalizedGini',
                              loss_function = 'Logloss',
                              n_estimators = 200,
                              max_depth = 5,
                              random_seed=42)


In [97]:
pipe_cat = Pipeline([('columns_transformer', columns_transformer),
                     ('catboost', CatBoostClassifier())])

pipe_cat.fit(X, y)

Learning rate set to 0.08499
0:	learn: 0.6926134	total: 13.5ms	remaining: 13.5s
1:	learn: 0.6921621	total: 23.6ms	remaining: 11.8s
2:	learn: 0.6918583	total: 34.4ms	remaining: 11.4s
3:	learn: 0.6914435	total: 44.9ms	remaining: 11.2s
4:	learn: 0.6913100	total: 55.6ms	remaining: 11.1s
5:	learn: 0.6911768	total: 65.8ms	remaining: 10.9s
6:	learn: 0.6910175	total: 75.5ms	remaining: 10.7s
7:	learn: 0.6909194	total: 86.3ms	remaining: 10.7s
8:	learn: 0.6908484	total: 96.2ms	remaining: 10.6s
9:	learn: 0.6905291	total: 106ms	remaining: 10.5s
10:	learn: 0.6899950	total: 116ms	remaining: 10.5s
11:	learn: 0.6898986	total: 127ms	remaining: 10.4s
12:	learn: 0.6898193	total: 138ms	remaining: 10.4s
13:	learn: 0.6894425	total: 148ms	remaining: 10.4s
14:	learn: 0.6893702	total: 158ms	remaining: 10.4s
15:	learn: 0.6892573	total: 169ms	remaining: 10.4s
16:	learn: 0.6891968	total: 179ms	remaining: 10.3s
17:	learn: 0.6891351	total: 192ms	remaining: 10.5s
18:	learn: 0.6890724	total: 205ms	remaining: 10.6s
19:

169:	learn: 0.6812540	total: 2s	remaining: 9.74s
170:	learn: 0.6811862	total: 2.01s	remaining: 9.74s
171:	learn: 0.6811509	total: 2.02s	remaining: 9.73s
172:	learn: 0.6811181	total: 2.03s	remaining: 9.72s
173:	learn: 0.6810417	total: 2.05s	remaining: 9.72s
174:	learn: 0.6809761	total: 2.06s	remaining: 9.7s
175:	learn: 0.6809239	total: 2.07s	remaining: 9.7s
176:	learn: 0.6808552	total: 2.08s	remaining: 9.69s
177:	learn: 0.6808119	total: 2.1s	remaining: 9.68s
178:	learn: 0.6807563	total: 2.11s	remaining: 9.68s
179:	learn: 0.6807285	total: 2.12s	remaining: 9.67s
180:	learn: 0.6806802	total: 2.13s	remaining: 9.66s
181:	learn: 0.6806270	total: 2.15s	remaining: 9.66s
182:	learn: 0.6805743	total: 2.16s	remaining: 9.65s
183:	learn: 0.6805065	total: 2.17s	remaining: 9.63s
184:	learn: 0.6804468	total: 2.18s	remaining: 9.62s
185:	learn: 0.6803766	total: 2.19s	remaining: 9.61s
186:	learn: 0.6803130	total: 2.21s	remaining: 9.6s
187:	learn: 0.6802499	total: 2.22s	remaining: 9.59s
188:	learn: 0.68020

335:	learn: 0.6722532	total: 3.82s	remaining: 7.54s
336:	learn: 0.6721954	total: 3.83s	remaining: 7.53s
337:	learn: 0.6721438	total: 3.84s	remaining: 7.52s
338:	learn: 0.6720961	total: 3.85s	remaining: 7.52s
339:	learn: 0.6720453	total: 3.87s	remaining: 7.51s
340:	learn: 0.6719967	total: 3.88s	remaining: 7.5s
341:	learn: 0.6719485	total: 3.89s	remaining: 7.49s
342:	learn: 0.6718995	total: 3.91s	remaining: 7.48s
343:	learn: 0.6718513	total: 3.92s	remaining: 7.47s
344:	learn: 0.6717935	total: 3.93s	remaining: 7.46s
345:	learn: 0.6717389	total: 3.94s	remaining: 7.44s
346:	learn: 0.6716917	total: 3.95s	remaining: 7.43s
347:	learn: 0.6716316	total: 3.96s	remaining: 7.42s
348:	learn: 0.6715716	total: 3.97s	remaining: 7.41s
349:	learn: 0.6715114	total: 3.98s	remaining: 7.39s
350:	learn: 0.6714590	total: 3.99s	remaining: 7.38s
351:	learn: 0.6714227	total: 4s	remaining: 7.37s
352:	learn: 0.6714007	total: 4.01s	remaining: 7.35s
353:	learn: 0.6713386	total: 4.02s	remaining: 7.34s
354:	learn: 0.67

502:	learn: 0.6640823	total: 6.06s	remaining: 5.99s
503:	learn: 0.6640382	total: 6.08s	remaining: 5.99s
504:	learn: 0.6640129	total: 6.1s	remaining: 5.98s
505:	learn: 0.6639703	total: 6.12s	remaining: 5.97s
506:	learn: 0.6639185	total: 6.13s	remaining: 5.96s
507:	learn: 0.6638778	total: 6.15s	remaining: 5.96s
508:	learn: 0.6638243	total: 6.17s	remaining: 5.95s
509:	learn: 0.6637795	total: 6.18s	remaining: 5.94s
510:	learn: 0.6637089	total: 6.2s	remaining: 5.93s
511:	learn: 0.6636592	total: 6.22s	remaining: 5.92s
512:	learn: 0.6636203	total: 6.23s	remaining: 5.92s
513:	learn: 0.6635442	total: 6.25s	remaining: 5.91s
514:	learn: 0.6635066	total: 6.27s	remaining: 5.9s
515:	learn: 0.6634621	total: 6.28s	remaining: 5.89s
516:	learn: 0.6634183	total: 6.3s	remaining: 5.88s
517:	learn: 0.6633670	total: 6.31s	remaining: 5.87s
518:	learn: 0.6633218	total: 6.33s	remaining: 5.87s
519:	learn: 0.6632636	total: 6.35s	remaining: 5.86s
520:	learn: 0.6632078	total: 6.36s	remaining: 5.85s
521:	learn: 0.66

662:	learn: 0.6565935	total: 8.55s	remaining: 4.35s
663:	learn: 0.6565607	total: 8.57s	remaining: 4.33s
664:	learn: 0.6565336	total: 8.58s	remaining: 4.32s
665:	learn: 0.6564799	total: 8.6s	remaining: 4.31s
666:	learn: 0.6564283	total: 8.61s	remaining: 4.3s
667:	learn: 0.6563849	total: 8.63s	remaining: 4.29s
668:	learn: 0.6563563	total: 8.64s	remaining: 4.27s
669:	learn: 0.6563282	total: 8.65s	remaining: 4.26s
670:	learn: 0.6562699	total: 8.66s	remaining: 4.25s
671:	learn: 0.6562074	total: 8.68s	remaining: 4.24s
672:	learn: 0.6561591	total: 8.69s	remaining: 4.22s
673:	learn: 0.6561142	total: 8.7s	remaining: 4.21s
674:	learn: 0.6560515	total: 8.72s	remaining: 4.2s
675:	learn: 0.6560256	total: 8.73s	remaining: 4.18s
676:	learn: 0.6559834	total: 8.74s	remaining: 4.17s
677:	learn: 0.6559400	total: 8.76s	remaining: 4.16s
678:	learn: 0.6559135	total: 8.77s	remaining: 4.15s
679:	learn: 0.6558642	total: 8.79s	remaining: 4.13s
680:	learn: 0.6558065	total: 8.8s	remaining: 4.12s
681:	learn: 0.655

832:	learn: 0.6489098	total: 11.1s	remaining: 2.22s
833:	learn: 0.6488555	total: 11.1s	remaining: 2.2s
834:	learn: 0.6488147	total: 11.1s	remaining: 2.19s
835:	learn: 0.6487739	total: 11.1s	remaining: 2.18s
836:	learn: 0.6487311	total: 11.1s	remaining: 2.16s
837:	learn: 0.6486910	total: 11.1s	remaining: 2.15s
838:	learn: 0.6486551	total: 11.1s	remaining: 2.14s
839:	learn: 0.6486180	total: 11.2s	remaining: 2.12s
840:	learn: 0.6485781	total: 11.2s	remaining: 2.11s
841:	learn: 0.6485465	total: 11.2s	remaining: 2.1s
842:	learn: 0.6485112	total: 11.2s	remaining: 2.08s
843:	learn: 0.6484763	total: 11.2s	remaining: 2.07s
844:	learn: 0.6484424	total: 11.2s	remaining: 2.06s
845:	learn: 0.6483869	total: 11.2s	remaining: 2.04s
846:	learn: 0.6483483	total: 11.2s	remaining: 2.03s
847:	learn: 0.6483237	total: 11.3s	remaining: 2.02s
848:	learn: 0.6482767	total: 11.3s	remaining: 2s
849:	learn: 0.6482310	total: 11.3s	remaining: 1.99s
850:	learn: 0.6481895	total: 11.3s	remaining: 1.98s
851:	learn: 0.648

Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22, 24,
                                                   25, 26, 27, 28, 29, 30, ...]),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  [23, 31, 32])])),
                ('catboost',
                 <catboost.core.CatBoostClassifier object at 0x7fe8dad903a0>)])

In [104]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()

pipe_log = Pipeline([('columns_transformer', columns_transformer),
                     ('log_reg', LogisticRegression())])

pipe_log.fit(X, y)

/Users/vadimbaev/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22, 24,
                                                   25, 26, 27, 28, 29, 30, ...]),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  [23, 31, 32])])),
                ('log_reg', LogisticRegression())])